In [2]:
import os
import numpy as np
import matplotlib

matplotlib.use('Agg')  # 使用Agg后端
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# base_path = '/content/drive/MyDrive/data'
base_path = '/Volumes/ExtData/workbench/2025/FLEA/Informer'
setting_path = base_path + '/results/informer_Normal_ftMS_sl96_ll48_pl24_dm512_nh8_el2_dl1_df2048_atprob_fc5_ebtimeF_dtTrue_mxTrue_exp_0'
font_path = base_path + '/Fonts/SIMSUN.TTC'
save_path = base_path + '/plots'


In [4]:

def plot_all_features_comparison(setting_path, sample_size, time_idx):
    """绘制所有特征的预测值与真实值对比图"""
    try:
        # 设置中文字体
        if os.path.exists(font_path):
            font = FontProperties(fname=font_path)
        else:  # 其他系统使用默认字体
            font = FontProperties()
            print("警告：未找到中文字体，将使用默认字体")

        # 加载数据
        pred_path = os.path.join(setting_path, 'pred.npy')
        true_path = os.path.join(setting_path, 'true.npy')

        if not os.path.exists(pred_path) or not os.path.exists(true_path):
            print("找不到预测结果文件")
            return

        pred = np.load(pred_path)
        true = np.load(true_path)

        print(f"数据形状: pred={pred.shape}, true={true.shape}")

        # 设置子图布局
        n_features = pred.shape[-1]  # 特征数量
        n_cols = 3  # 每行显示3个子图
        n_rows = (n_features + n_cols - 1) // n_cols  # 计算需要的行数

        # 创建大图和子图
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 5 * n_rows))
        axes = axes.flatten()  # 将axes数组展平，便于索引

        # 选择要展示的样本数量和时间步
        sample_size = sample_size
        time_idx = time_idx

        # 计算总体评估指标
        mae = np.mean(np.abs(pred - true))
        mse = np.mean((pred - true) ** 2)
        rmse = np.sqrt(mse)

        # 特征名称列表（根据您的数据集调整）
        feature_names = [
            '能见度', 'DryBulbFarenheit', 'DryBulbCelsius', 'WetBulbFarenheit',
            'DewPointFarenheit', 'DewPointCelsius', 'RelativeHumidity', '风速',
            '风向', '测站气压', '测高仪', 'WetBulbCelsius'
        ]

        # 在每个子图中绘制对比图
        for i in range(n_features):
            ax = axes[i]

            # 获取当前特征的预测值和真实值
            pred_values = pred[:sample_size, time_idx, i]
            true_values = true[:sample_size, time_idx, i]

            # 绘制对比图
            ax.plot(true_values, 'b-', label='真实值', linewidth=1.5)
            ax.plot(pred_values, 'r--', label='预测值', linewidth=1.5)

            # 计算当前特征的评估指标
            feature_mae = np.mean(np.abs(pred[:, :, i] - true[:, :, i]))
            feature_rmse = np.sqrt(np.mean((pred[:, :, i] - true[:, :, i]) ** 2))

            # 设置子图标题和标签（使用fontproperties指定字体）
            ax.set_title(f'{feature_names[i]}\nMAE: {feature_mae:.4f}\nRMSE: {feature_rmse:.4f}',
                         fontproperties=font)
            ax.set_xlabel('样本索引', fontproperties=font)
            ax.set_ylabel('数值', fontproperties=font)
            ax.legend(prop=font)
            ax.grid(True)

        # 隐藏多余的子图
        for i in range(n_features, len(axes)):
            axes[i].set_visible(False)

        # 添加总体评估指标作为大标题
        plt.suptitle('所有特征的预测值与真实值对比\n' +
                     f'总体评估指标 - MAE: {mae:.4f}, MSE: {mse:.4f}, RMSE: {rmse:.4f}',
                     fontproperties=font, fontsize=16, y=1.02)

        # 调整子图之间的间距
        plt.tight_layout()

        # 保存图表
        save_dir = save_path
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        plt.savefig(os.path.join(save_dir, 'all_features_comparison.png'),
                    dpi=300,
                    bbox_inches='tight')
        plt.close()

        # 打印评估指标
        print('\n总体评估指标:')
        print(f'MAE: {mae:.4f}')
        print(f'MSE: {mse:.4f}')
        print(f'RMSE: {rmse:.4f}')

    except Exception as e:
        print(f"发生错误: {str(e)}")
        plt.close()

In [5]:

def plot_prediction_comparison(setting_path, sample_size, feature_idx, time_idx):
    """在一张图中展示预测值和真实值的对比"""
    try:
        # 设置中文字体
        if os.path.exists(font_path):
            font = FontProperties(fname=font_path)
        else:  # 其他系统使用默认字体
            font = FontProperties()
            print("警告：未找到中文字体，将使用默认字体")

        # 加载数据
        pred_path = os.path.join(setting_path, 'pred.npy')
        true_path = os.path.join(setting_path, 'true.npy')

        if not os.path.exists(pred_path) or not os.path.exists(true_path):
            print("找不到预测结果文件")
            return

        pred = np.load(pred_path)
        true = np.load(true_path)

        print(f"数据形状: pred={pred.shape}, true={true.shape}")

        # 创建新图表
        plt.figure(figsize=(15, 8))

        # 选择要展示的数据
        sample_size = sample_size  # 样本数量
        feature_idx = feature_idx  # 特征索引
        time_idx = time_idx  # 时间步索引

        pred_values = pred[:sample_size, time_idx, feature_idx]
        true_values = true[:sample_size, time_idx, feature_idx]

        # 绘制对比图
        plt.plot(true_values, 'b-', label=u'真实值', linewidth=2)
        plt.plot(pred_values, 'r--', label=u'预测值', linewidth=2)

        # 设置标题和标签（使用fontproperties指定中文字体）
        plt.title(u'预测结果与真实值对比', fontproperties=font, fontsize=14)
        plt.xlabel(u'样本索引', fontproperties=font)
        plt.ylabel(u'数值', fontproperties=font)
        plt.legend(prop=font)
        plt.grid(True)

        # 计算评估指标
        mae = np.mean(np.abs(pred - true))  # 平均绝对误差
        mse = np.mean((pred - true) ** 2)  # 均方误差
        rmse = np.sqrt(mse)  # 均方根误差

        # 添加评估指标文本
        metrics_text = f'评估指标:\nMAE: {mae:.4f}\nMSE: {mse:.4f}\nRMSE: {rmse:.4f}'
        plt.text(0.02, 0.98, metrics_text,
                 transform=plt.gca().transAxes,
                 bbox=dict(facecolor='white', alpha=0.8),
                 verticalalignment='top',
                 fontproperties=font)

        # 保存图表
        save_dir = save_path
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        plt.savefig(os.path.join(save_dir, 'prediction_comparison.png'),
                    dpi=300,
                    bbox_inches='tight',
                    format='png')
        plt.close()

        # 打印评估指标
        print('\n评估指标:')
        print(f'MAE: {mae:.4f}')
        print(f'MSE: {mse:.4f}')
        print(f'RMSE: {rmse:.4f}')

    except Exception as e:
        print(f"发生错误: {str(e)}")
        plt.close()

In [ ]:
matplotlib.use('TkAgg')  # 改用Agg后端

#绘制单个特征的预测值与真实值对比图，sample_size为样本数量，feature_idx为特征索引，time_idx为时间步索引
plot_prediction_comparison(setting_path, sample_size=200, feature_idx=1, time_idx=0)

In [6]:
#matplotlib.use('TkAgg')  # 改用Agg后端
matplotlib.use('Agg')  # 改用Agg后端

#绘制所有特征的预测值与真实值对比图，sample_size为样本数量，time_idx为时间步索引
plot_all_features_comparison(setting_path, sample_size=500, time_idx=0)

数据形状: pred=(8288, 24, 1), true=(8288, 24, 1)

总体评估指标:
MAE: 0.1285
MSE: 0.0269
RMSE: 0.1641
